In [1]:
import findspark
findspark.init('/usr/local/spark')
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("capstone checkpoint").getOrCreate()

In [3]:
df = spark.read.csv("user_demographics.csv",inferSchema=True,header=True)

In [4]:
df1 = spark.read.csv("User_product_purchase_details_p2.csv",inferSchema=True,header=True)

In [5]:
df.show(10)

+-------+------+-----+----------+
|User_ID|Gender|  Age|Occupation|
+-------+------+-----+----------+
|1000001|     F| 0-17|        10|
|1000002|     M|  55+|        16|
|1000003|     M|26-35|        15|
|1000004|     M|46-50|         7|
|1000005|     M|26-35|        20|
|1000006|     F|51-55|         9|
|1000007|     M|36-45|         1|
|1000008|     M|26-35|        12|
|1000009|     M|26-35|        17|
|1000010|     F|36-45|         1|
+-------+------+-----+----------+
only showing top 10 rows



In [6]:
df1.show(10)

+-------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|            A|                         2|             0|                12|              null|              null|    1422|
|1000001| P00085442|            A|                         2|             0|                12|                14|              null|    1057|

In [7]:
output = df.join(df1,['User_ID'],how='inner')

In [8]:
output.show(5)

+-------+------+----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Gender| Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+------+----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001|     F|0-17|        10| P00069042|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001|     F|0-17|        10| P00248942|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001|     F|0-17|        10| P00087842|            A|                         2|             0|                12|              null|              null|    1422|
|100

In [9]:
output.drop(output.Product_Category_3).show()

+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|User_ID|Gender|  Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|1000001|     F| 0-17|        10| P00069042|            A|                         2|             0|                 3|              null|    8370|
|1000001|     F| 0-17|        10| P00248942|            A|                         2|             0|                 1|                 6|   15200|
|1000001|     F| 0-17|        10| P00087842|            A|                         2|             0|                12|              null|    1422|
|1000001|     F| 0-17|        10| P00085442|            A|                         2|             0|            

In [10]:
output1 = output.drop(output.Product_Category_3).fillna(value=0,subset=["Product_Category_2"])

In [11]:
spark.sql("create database blackfriday").show()

++
||
++
++



In [12]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
| blackfriday|
|     default|
+------------+



In [13]:
spark.sql("use blackfriday").show()

++
||
++
++



In [14]:
output1.write.saveAsTable('black_friday1')

In [15]:
output1.describe().show()

+-------+------------------+------+------+------------------+----------+-------------+--------------------------+-------------------+------------------+------------------+-----------------+
|summary|           User_ID|Gender|   Age|        Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|     Marital_Status|Product_Category_1|Product_Category_2|         Purchase|
+-------+------------------+------+------+------------------+----------+-------------+--------------------------+-------------------+------------------+------------------+-----------------+
|  count|            550068|550068|550068|            550068|    550068|       550068|                    550068|             550068|            550068|            550068|           550068|
|   mean|1003028.8424013031|  null|  null| 8.076706879876669|      null|         null|         1.468494139793958|0.40965298835780306| 5.404270017525106| 6.735436346051761|9263.968712959126|
| stddev| 1727.591585530871|  null|  null|6.522660

In [16]:
output1.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [17]:
spark.sql("show tables").show()

+-----------+-------------+-----------+
|   database|    tableName|isTemporary|
+-----------+-------------+-----------+
|blackfriday|black_friday1|      false|
+-----------+-------------+-----------+



In [18]:
spark.sql("select * from black_friday1 limit 10").show()

+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|User_ID|Gender|  Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|1000001|     F| 0-17|        10| P00069042|            A|                         2|             0|                 3|                 0|    8370|
|1000001|     F| 0-17|        10| P00248942|            A|                         2|             0|                 1|                 6|   15200|
|1000001|     F| 0-17|        10| P00087842|            A|                         2|             0|                12|                 0|    1422|
|1000001|     F| 0-17|        10| P00085442|            A|                         2|             0|            

In [19]:
output1[output1.Purchase > 5000].show()

+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|User_ID|Gender|  Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|1000001|     F| 0-17|        10| P00069042|            A|                         2|             0|                 3|                 0|    8370|
|1000001|     F| 0-17|        10| P00248942|            A|                         2|             0|                 1|                 6|   15200|
|1000002|     M|  55+|        16| P00285442|            C|                        4+|             0|                 8|                 0|    7969|
|1000003|     M|26-35|        15| P00193542|            A|                         3|             0|            

In [20]:
output1.groupby("Gender").count().show()

+------+------+
|Gender| count|
+------+------+
|     F|135809|
|     M|414259|
+------+------+



In [21]:
output1.groupby("City_Category").count().show()

+-------------+------+
|City_Category| count|
+-------------+------+
|            B|231173|
|            C|171175|
|            A|147720|
+-------------+------+



In [22]:
spark.sql("select * from black_friday1 where Gender='M' and City_Category='A'").count()

112016

In [23]:
spark.sql("select * from black_friday1 where Gender='M' and City_Category='A'").show()

+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|User_ID|Gender|  Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|1000003|     M|26-35|        15| P00193542|            A|                         3|             0|                 1|                 2|   15227|
|1000005|     M|26-35|        20| P00274942|            A|                         1|             1|                 8|                 0|    7871|
|1000005|     M|26-35|        20| P00251242|            A|                         1|             1|                 5|                11|    5254|
|1000005|     M|26-35|        20| P00014542|            A|                         1|             1|            

In [24]:
output1.show(10)

+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|User_ID|Gender|  Age|Occupation|Product_ID|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+-------+------+-----+----------+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|1000001|     F| 0-17|        10| P00069042|            A|                         2|             0|                 3|                 0|    8370|
|1000001|     F| 0-17|        10| P00248942|            A|                         2|             0|                 1|                 6|   15200|
|1000001|     F| 0-17|        10| P00087842|            A|                         2|             0|                12|                 0|    1422|
|1000001|     F| 0-17|        10| P00085442|            A|                         2|             0|            

In [25]:
output = output1.drop('User_ID')

In [26]:
output = output.drop('Product_ID')

In [27]:
output.show(5)

+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|
+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+--------+
|     F|0-17|        10|            A|                         2|             0|                 3|                 0|    8370|
|     F|0-17|        10|            A|                         2|             0|                 1|                 6|   15200|
|     F|0-17|        10|            A|                         2|             0|                12|                 0|    1422|
|     F|0-17|        10|            A|                         2|             0|                12|                14|    1057|
|     M| 55+|        16|            C|                        4+|             0|                 8|     

In [28]:
from pyspark.ml.feature import StringIndexer
city_indexer = StringIndexer(inputCol="City_Category", outputCol="CityCategory")
output = city_indexer.fit(output).transform(output)

In [29]:
output.show(5)

+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+--------+------------+
|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|CityCategory|
+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+--------+------------+
|     F|0-17|        10|            A|                         2|             0|                 3|                 0|    8370|         2.0|
|     F|0-17|        10|            A|                         2|             0|                 1|                 6|   15200|         2.0|
|     F|0-17|        10|            A|                         2|             0|                12|                 0|    1422|         2.0|
|     F|0-17|        10|            A|                         2|             0|                12|                14|    1057|         2.0|
|     M| 55+|

In [30]:
output = output.drop('City_Category')

In [31]:
age_indexer = StringIndexer(inputCol="Age", outputCol="ageindex")
output = age_indexer.fit(output).transform(output)
output.show(5)

+------+----+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+
|Gender| Age|Occupation|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|CityCategory|ageindex|
+------+----+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+
|     F|0-17|        10|                         2|             0|                 3|                 0|    8370|         2.0|     6.0|
|     F|0-17|        10|                         2|             0|                 1|                 6|   15200|         2.0|     6.0|
|     F|0-17|        10|                         2|             0|                12|                 0|    1422|         2.0|     6.0|
|     F|0-17|        10|                         2|             0|                12|                14|    1057|         2.0|     6.0|
|     M| 55+|        16|                        

In [32]:
output = output.drop('Age')

In [33]:
gender_indexer = StringIndexer(inputCol="Gender", outputCol="genderindex")
output = gender_indexer.fit(output).transform(output)
output.show(5)

+------+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+-----------+
|Gender|Occupation|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|CityCategory|ageindex|genderindex|
+------+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+-----------+
|     F|        10|                         2|             0|                 3|                 0|    8370|         2.0|     6.0|        1.0|
|     F|        10|                         2|             0|                 1|                 6|   15200|         2.0|     6.0|        1.0|
|     F|        10|                         2|             0|                12|                 0|    1422|         2.0|     6.0|        1.0|
|     F|        10|                         2|             0|                12|                14|    1057|         2.0|     6.0|        1.0|

In [34]:
output = output.drop('Gender')

In [35]:
stay_indexer = StringIndexer(inputCol="Stay_In_Current_City_Years", outputCol="stayindex")
output = stay_indexer.fit(output).transform(output)
output.show(10)

+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+-----------+---------+
|Occupation|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Purchase|CityCategory|ageindex|genderindex|stayindex|
+----------+--------------------------+--------------+------------------+------------------+--------+------------+--------+-----------+---------+
|        10|                         2|             0|                 3|                 0|    8370|         2.0|     6.0|        1.0|      1.0|
|        10|                         2|             0|                 1|                 6|   15200|         2.0|     6.0|        1.0|      1.0|
|        10|                         2|             0|                12|                 0|    1422|         2.0|     6.0|        1.0|      1.0|
|        10|                         2|             0|                12|                14|    1057|         2.0|     6.0| 

In [36]:
output = output.drop('Stay_In_Current_City_Years')

In [37]:
output.show(5)

+----------+--------------+------------------+------------------+--------+------------+--------+-----------+---------+
|Occupation|Marital_Status|Product_Category_1|Product_Category_2|Purchase|CityCategory|ageindex|genderindex|stayindex|
+----------+--------------+------------------+------------------+--------+------------+--------+-----------+---------+
|        10|             0|                 3|                 0|    8370|         2.0|     6.0|        1.0|      1.0|
|        10|             0|                 1|                 6|   15200|         2.0|     6.0|        1.0|      1.0|
|        10|             0|                12|                 0|    1422|         2.0|     6.0|        1.0|      1.0|
|        10|             0|                12|                14|    1057|         2.0|     6.0|        1.0|      1.0|
|        16|             0|                 8|                 0|    7969|         1.0|     5.0|        0.0|      3.0|
+----------+--------------+------------------+--

In [38]:
features = ['Occupation','stayindex','Marital_Status','Product_Category_1','Product_Category_2','CityCategory','ageindex','genderindex']

In [40]:
from pyspark.sql.functions import *
lr_data = output.select(col("Purchase").alias("label"), *features)
lr_data.printSchema()

root
 |-- label: integer (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- stayindex: double (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- CityCategory: double (nullable = true)
 |-- ageindex: double (nullable = true)
 |-- genderindex: double (nullable = true)



In [41]:
lr_data.show(10)

+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+
|label|Occupation|stayindex|Marital_Status|Product_Category_1|Product_Category_2|CityCategory|ageindex|genderindex|
+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+
| 8370|        10|      1.0|             0|                 3|                 0|         2.0|     6.0|        1.0|
|15200|        10|      1.0|             0|                 1|                 6|         2.0|     6.0|        1.0|
| 1422|        10|      1.0|             0|                12|                 0|         2.0|     6.0|        1.0|
| 1057|        10|      1.0|             0|                12|                14|         2.0|     6.0|        1.0|
| 7969|        16|      3.0|             0|                 8|                 0|         1.0|     5.0|        0.0|
|15227|        15|      2.0|             0|                 1|          

In [42]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [43]:
vectorAssembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")

In [44]:
va_data = vectorAssembler.transform(lr_data)

In [45]:
va_data.show(truncate=False)

+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+------------------------------------+
|label|Occupation|stayindex|Marital_Status|Product_Category_1|Product_Category_2|CityCategory|ageindex|genderindex|unscaled_features                   |
+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+------------------------------------+
|8370 |10        |1.0      |0             |3                 |0                 |2.0         |6.0     |1.0        |[10.0,1.0,0.0,3.0,0.0,2.0,6.0,1.0]  |
|15200|10        |1.0      |0             |1                 |6                 |2.0         |6.0     |1.0        |[10.0,1.0,0.0,1.0,6.0,2.0,6.0,1.0]  |
|1422 |10        |1.0      |0             |12                |0                 |2.0         |6.0     |1.0        |[10.0,1.0,0.0,12.0,0.0,2.0,6.0,1.0] |
|1057 |10        |1.0      |0             |12                |14                |2

In [46]:
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

In [47]:
ss_model = standardScaler.fit(va_data)

In [48]:
ss_data = ss_model.transform(va_data)

In [49]:
ss_data.show(truncate=False)

+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|label|Occupation|stayindex|Marital_Status|Product_Category_1|Product_Category_2|CityCategory|ageindex|genderindex|unscaled_features                   |features                                                                                                                               |
+-----+----------+---------+--------------+------------------+------------------+------------+--------+-----------+------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------+
|8370 |10        |1.0      |0             |3                 |0                 |2.0         |6.0     |1.0        |[10.0,1.0,0.0,3.0,

## Splitting tranning and testing data

In [50]:
(training, test) = ss_data.randomSplit([.7, .3])
training.describe().show()

+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|            label|       Occupation|         stayindex|    Marital_Status|Product_Category_1|Product_Category_2|      CityCategory|          ageindex|        genderindex|
+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|           385050|           385050|            385050|            385050|            385050|            385050|            385050|            385050|             385050|
|   mean|  9265.2289105311|8.076351123230749|1.5336605635631737|0.4097545773276198| 5.405396701727049|  6.74067523698221|0.8483080119465004|1.4545072068562523|0.24691338787170497|
| stddev|5024.740284033693|6.520155213782972|1.4398959501717996| 0.491788970803052| 3.93354874909221

In [51]:
test.describe().show()

+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|summary|            label|       Occupation|         stayindex|    Marital_Status|Product_Category_1|Product_Category_2|      CityCategory|          ageindex|        genderindex|
+-------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+
|  count|           165018|           165018|            165018|            165018|            165018|            165018|            165018|            165018|             165018|
|   mean|9261.028190864026|8.077536995964076| 1.537056563526403|0.4094159425032421|  5.40164103309942|  6.72321201323492|0.8482347380285787|1.4453817159340194|0.24685185858512404|
| stddev|5019.169049452351|6.528522198968895|1.4396520306606073|0.4917275605432764|3.942427962146734

## LinearRegression

In [53]:
lr = LinearRegression(maxIter=10, regParam=.01)
lr_model = lr.fit(training)

In [54]:
lr_model.coefficients

DenseVector([62.7314, -15.0926, 28.4164, -1707.9995, 141.8912, -32.3229, 53.789, -219.5719])

In [55]:
lr_model.intercept

11482.858598621868

In [56]:
trainingSummary = lr_model.summary
trainingSummary.rootMeanSquaredError

4710.652264556437

In [57]:
trainingSummary.r2

0.12110704719571241

In [58]:
prediction_df = lr_model.transform(test)

In [59]:
prediction_df.select("label","prediction").show(truncate=False)

+-----+------------------+
|label|prediction        |
+-----+------------------+
|12   |2796.8525794167836|
|12   |3167.324908081482 |
|12   |3282.4280975365964|
|12   |3305.7863193030125|
|12   |3288.7305657124907|
|12   |3333.845475943621 |
|12   |3303.3210655681987|
|12   |3332.4519936284423|
|12   |3261.3919429848247|
|12   |2716.689749105273 |
|12   |3368.4065122688226|
|12   |3275.1251634729633|
|12   |3365.2131843493826|
|12   |3485.1677477423555|
|12   |2963.015468706686 |
|12   |3461.7379327158533|
|12   |3337.6035568529333|
|12   |3326.2564565678367|
|12   |3373.5580754933126|
|12   |3424.239839444028 |
+-----+------------------+
only showing top 20 rows



In [60]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

In [61]:
rmse = eval.evaluate(prediction_df)
print("RMSE: %.3f" % rmse)
r2 = eval.evaluate(prediction_df, {eval.metricName: "r2"})
print("r2: %.3f" %r2)
mse = eval.evaluate(prediction_df, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
mae = eval.evaluate(prediction_df, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

RMSE: 4703.250
r2: 0.122
MSE: 22120561.809
MAE: 3601.879


## DecisionTreeRegressor

In [62]:
from pyspark.ml.regression import DecisionTreeRegressor

In [66]:
dtr = DecisionTreeRegressor(maxDepth=15)

In [68]:
dtr_model = dtr.fit(training)

In [70]:
prediction_dtr = dtr_model.transform(test)

In [72]:
rmse = eval.evaluate(prediction_dtr)
print("RMSE: %.3f" % rmse)
r2 = eval.evaluate(prediction_dtr, {eval.metricName: "r2"})
print("r2: %.3f" %r2)
mse = eval.evaluate(prediction_dtr, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
mae = eval.evaluate(prediction_dtr, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

RMSE: 3019.575
r2: 0.638
MSE: 9117831.837
MAE: 2233.796


## RandomForestRegressor

In [73]:
from pyspark.ml.regression import RandomForestRegressor

In [74]:
rfr = RandomForestRegressor(maxDepth=15)

In [75]:
rfr_model = rfr.fit(training)

In [76]:
prediction_rfr = rfr_model.transform(test)

In [77]:
rmse = eval.evaluate(prediction_rfr)
print("RMSE: %.3f" % rmse)
r2 = eval.evaluate(prediction_rfr, {eval.metricName: "r2"})
print("r2: %.3f" %r2)
mse = eval.evaluate(prediction_rfr, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
mae = eval.evaluate(prediction_rfr, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

RMSE: 3001.737
r2: 0.642
MSE: 9010423.574
MAE: 2268.147


## Gradient-boosted tree regression

In [78]:
from pyspark.ml.regression import GBTRegressor

In [79]:
gbt = GBTRegressor(maxIter=10)

In [80]:
gbt_model = gbt.fit(training)

In [81]:
prediction_gbt = gbt_model.transform(test)

In [82]:
rmse = eval.evaluate(prediction_gbt)
print("RMSE: %.3f" % rmse)
r2 = eval.evaluate(prediction_gbt, {eval.metricName: "r2"})
print("r2: %.3f" %r2)
mse = eval.evaluate(prediction_gbt, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)
mae = eval.evaluate(prediction_gbt, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

RMSE: 3003.840
r2: 0.642
MSE: 9023057.483
MAE: 2282.940
